In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import json
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from glob import glob
import re 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import gc
import geopandas
from shapely import wkt

In [ ]:
"""
CITY DATA
"""
Full_Cities_Dataset = pd.concat([pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2018_Full_Cities_Dataset.csv'),
                                 pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2019_Full_Cities_Dataset.csv'),
                                 pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv')
                                 ])
Cities_Disclosing_to_CDP = pd.concat([pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv'),
                                      pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2019_Cities_Disclosing_to_CDP.csv'),
                                      pd.read_csv('../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2018_Cities_Disclosing_to_CDP.csv')
                                      ])

cities_data = pd.merge(Full_Cities_Dataset[['Account Number','Parent Section', 'Section',
                       'Question Number', 'Question Name', 'Column Number', 'Column Name', 
                       'Row Number', 'Row Name', 'Response Answer', 'Comments']],
                       Cities_Disclosing_to_CDP[Cities_Disclosing_to_CDP['Account Number'].isin(Full_Cities_Dataset['Account Number'])],
                       on='Account Number',
                       how='left'
                       )
"""
CORPORATION DATA
"""
Corporates_Disclosing_to_CDP_Water_Security = pd.concat([pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Disclosing/Water Security/2020_Corporates_Disclosing_to_CDP_Water_Security.csv'), 
                                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Disclosing/Water Security/2019_Corporates_Disclosing_to_CDP_Water_Security.csv'),
                                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Disclosing/Water Security/2018_Corporates_Disclosing_to_CDP_Water_Security.csv')
                                                         ])
Corporates_Disclosing_to_CDP_Climate_Change = pd.concat([pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Disclosing/Climate Change/2018_Corporates_Disclosing_to_CDP_Climate_Change.csv'),
                                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Disclosing/Climate Change/2019_Corporates_Disclosing_to_CDP_Climate_Change.csv'),
                                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Disclosing/Climate Change/2020_Corporates_Disclosing_to_CDP_Climate_Change.csv')
                                                         ])

Full_Water_Security_Dataset = pd.concat([pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Water Security/2019_Full_Water_Security_Dataset.csv'),
                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Water Security/2018_Full_Water_Security_Dataset.csv'),
                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Water Security/2020_Full_Water_Security_Dataset.csv')
                                         ])

Full_Climate_Change_Dataset = pd.concat([pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Climate Change/2019_Full_Climate_Change_Dataset.csv'),
                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Climate Change/2018_Full_Climate_Change_Dataset.csv'),
                                         pd.read_csv('../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Climate Change/2020_Full_Climate_Change_Dataset.csv',)
                                         ])

Corporates_Disclosing_to_CDP = pd.concat([Corporates_Disclosing_to_CDP_Climate_Change,
                                          Corporates_Disclosing_to_CDP_Water_Security[~Corporates_Disclosing_to_CDP_Water_Security.account_number.isin(Corporates_Disclosing_to_CDP_Climate_Change.account_number)]
                                          ])

Corporates_Climate_Water_Dataset = pd.concat([Full_Climate_Change_Dataset,
                                              Full_Water_Security_Dataset
                                ])
"""
SUPPLEMENTARY DATA
"""
Social_Vulnerability_Index_2018_county =  pd.read_csv('../input/cdp-unlocking-climate-solutions/Supplementary Data/CDC Social Vulnerability Index 2018/SVI2018_US_COUNTY.csv',)
svi = pd.read_csv('../input/cdp-unlocking-climate-solutions/Supplementary Data/CDC Social Vulnerability Index 2018/SVI2018_US.csv', usecols=['STATE', 'COUNTY', 'AREA_SQMI', 
                                       'E_TOTPOP', 'EP_POV', 'EP_UNEMP', 'EP_PCI', 'EP_NOHSDP', 'EP_MINRTY', 'EP_AGE65', 'EP_AGE17', 
                          'EP_DISABL', 'EP_SNGPNT', 'EP_LIMENG', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'RPL_THEMES'])
Cities_Census = pd.read_csv('../input/cdp-unlocking-climate-solutions/Supplementary Data/CDC 500 Cities Census Tract Data/500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release.csv',)
US_cities_map = pd.read_csv('../input/cdp-unlocking-climate-solutions/Supplementary Data/Simple Maps US Cities Data/uscities.csv',usecols=['city', 'state_name', 'county_fips', 'county_name', 'lat', 'lng', 'population', 'density', 'source', 'military', 'incorporated', 'ranking'])
Locations = pd.read_csv('../input/cdp-unlocking-climate-solutions/Supplementary Data/Locations of Corporations/NA_HQ_public_data.csv')
Locations.drop(columns={'address_lat', 'adress_long', 'survey_name'}, inplace=True)
us_state_abbrev = {
    'AB':'ALBERTA',
    'AL':'Alabama',
    'AK':'Alaska',
    'AS':'American Samoa',
    'AZ':'Arizona',
    'AR':'Arkansas',
    'CA':'California',
    'CO':'Colorado',
    'CT':'Connecticut',
    'DE':'Delaware',
    'DC':'District of Columbia',
    'FL':'Florida',
    'GA':'Georgia',
    'GU':'Guam',
    'HI':'Hawaii',
    'ID':'Idaho',
    'IL':'Illinois',
    'IN':'Indiana',
    'IA':'Iowa',
    'KS':'Kansas',
    'KY':'Kentucky',
    'LA':'Louisiana',
    'ME':'Maine',
    'MD':'Maryland',
    'MA':'Massachusetts',
    'MI':'Michigan',
    'MN':'Minnesota',
    'MS':'Mississippi',
    'MO':'Missouri',
    'MT':'Montana',
    'NE':'Nebraska',
    'NV':'Nevada',
    'NV':'New Hampshire',
    'NV':'New Jersey',
    'NM':'New Mexico',
    'NY':'New York',
    'NC':'North Carolina',
    'ND':'North Dakota',
    'MP':'Northern Mariana Islands',
    'OH':'Ohio',
    'OK':'Oklahoma',
    'OR':'Oregon',
    'PA':'Pennsylvania',
    'PR':'Puerto Rico',
    'RI':'Rhode Island',
    'SC':'South Carolina',
    'SD':'South Dakota',
    'TN':'Tennessee',
    'TX':'Texas',
    'UT':'Utah',
    'VT':'Vermont',
    'VI':'Virgin Islands',
    'VA':'Virginia',
    'WA':'Washington',
    'WV':'West Virginia',
    'WI':'Wisconsin',
    'WY':'Wyoming'
}

Cities_Census['StateAbbr'] = Cities_Census['StateAbbr'].map(us_state_abbrev)

cities_data = cities_data[~cities_data['Response Answer'].isnull()]

svi_unique = pd.DataFrame([svi[svi.STATE==x].values[0] for x in svi.STATE.unique()], columns=svi.columns)[['STATE']]
svi_state = svi[['STATE', 'AREA_SQMI', 'E_TOTPOP', 'EP_POV', 'EP_UNEMP', 'EP_PCI', 'EP_NOHSDP', 'EP_MINRTY', 'EP_AGE65', 'EP_AGE17', 
                       'EP_DISABL', 'EP_SNGPNT', 'EP_LIMENG', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 
                       'RPL_THEMES']].groupby('STATE').agg('sum')                                                      
svi_state = pd.merge(svi_state, svi_unique, on='STATE', how='left')
svi_state.STATE = svi_state.STATE.str.capitalize()

census_df = pd.merge(Cities_Census, US_cities_map, left_on=['StateAbbr','PlaceName'], right_on=['state_name','city'], how='left')
census_df.rename(columns={'PlaceFIPS':'city_fips'}, inplace=True)
census_df.drop(columns={'StateAbbr', 'PlaceName'}, inplace=True)
census = census_df[['city', 'Population2010', 'ACCESS2_CrudePrev' , 'ARTHRITIS_CrudePrev', 'BPHIGH_CrudePrev', 'CANCER_CrudePrev', 
                     'CASTHMA_CrudePrev', 'CHD_CrudePrev', 'CHECKUP_CrudePrev', 'COLON_SCREEN_CrudePrev', 'DIABETES_CrudePrev', 
                     'KIDNEY_CrudePrev', 'LPA_CrudePrev', 'MHLTH_CrudePrev', 'OBESITY_CrudePrev', 'PHLTH_CrudePrev', 'SLEEP_CrudePrev',
                     'STROKE_CrudePrev']] 
census = census.groupby('city').agg('mean')
census_unique = pd.DataFrame([census_df[census_df.city==x].values[0] for x in census_df.city.unique() if len(census_df[census_df.city==x].values)>0], 
                             columns=census_df.columns)[['city_fips', 'city', 'state_name', 'lat', 'lng', 'population', 'density', 'source', 'military',
                                                      'incorporated', 'ranking']]
census = pd.merge(census, census_unique, on='city', how='left')
census = census[~census.city.isnull()]

org = pd.merge(Locations, US_cities_map, left_on='address_city', right_on='city', how='left')
org.dropna(inplace=True)

### Some Numbers From The Data

#### City Data

In [ ]:
print('\nCDP Regions:', cities_data['CDP Region'].nunique())
print('Parent Sections:', cities_data['Parent Section'].nunique())
print('Sections:', cities_data['Section'].nunique())
print('Question Numbers:', cities_data['Question Number'].nunique())
print('Questions:', cities_data['Question Name'].nunique())
print('Columns:', cities_data['Column Name'].nunique())
print('Rows:', cities_data['Row Name'].nunique())
print('Account Numbers:', cities_data['Account Number'].nunique())
print('Organizations:', cities_data['Organization'].nunique())
print('Cities:', cities_data['City'].nunique())
print('Countries:', cities_data['Country'].nunique())
print('CDP Regions:', cities_data['CDP Region'].nunique())
print('Reporting Authorities:', cities_data['Reporting Authority'].nunique())
print('Access Types:', cities_data['Access'].nunique())
print('Account Numbers:', cities_data['Account Number'].nunique())

#### Corporations Data

In [ ]:
print('Number of organizations:',Corporates_Climate_Water_Dataset.organization.nunique())
print('Modules:',Corporates_Climate_Water_Dataset.module_name.nunique())
print('Questions:',Corporates_Climate_Water_Dataset.question_number.nunique())
print('Columns',Corporates_Climate_Water_Dataset.column_name.nunique())
print('Rows:',Corporates_Climate_Water_Dataset.row_name.nunique())

In [ ]:
del Full_Cities_Dataset 
del Cities_Disclosing_to_CDP 
del Social_Vulnerability_Index_2018_county 
del Corporates_Disclosing_to_CDP_Water_Security
del Corporates_Disclosing_to_CDP_Climate_Change
del Full_Water_Security_Dataset
del Full_Climate_Change_Dataset
gc.collect()

### Functions used for automated insight generation

In [ ]:
columns = ['ACCESS2_CrudePrev',	'ARTHRITIS_CrudePrev', 'BPHIGH_CrudePrev', 'CANCER_CrudePrev', 'CASTHMA_CrudePrev',	'CHD_CrudePrev',	'CHECKUP_CrudePrev',	'COLON_SCREEN_CrudePrev',	'DIABETES_CrudePrev', 'KIDNEY_CrudePrev', 'LPA_CrudePrev', 'MHLTH_CrudePrev', 'OBESITY_CrudePrev',	'PHLTH_CrudePrev', 'SLEEP_CrudePrev', 'STROKE_CrudePrev']

"""
Used to aggregate data over different years (2018, 2019, 2020), and plot the final data to make a comparision over the years.
"""
def corp_wc_yr(cn,mn,xlabel,title,rename_18,rename_19,rename_20,annotate=True):
  data = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name==mn][['organization','survey_year','response_value','module_name','column_name']]
  df = data[data.column_name==cn]
  df.dropna(inplace=True)
  df['response_value'] = df['response_value'].astype('float')
  
  if df.survey_year.unique().__contains__(2018):
    df_agg = df[df.survey_year==2018][['organization', 'response_value']]
    df_agg = df_agg.groupby(['organization']).agg('mean')
    df_agg.rename(columns={'response_value':rename_18}, inplace=True)

  if df.survey_year.unique().__contains__(2019):
    df_agg[rename_19] = df[df.survey_year==2019][['organization', 'response_value']].groupby('organization').agg('mean')

  if df.survey_year.unique().__contains__(2020):  
    df_agg[rename_20] = df[df.survey_year==2020][['organization', 'response_value']].groupby('organization').agg('mean')
  
  if df.survey_year.unique().max()==2020:
    max_year=rename_20
  elif df.survey_year.unique().max()==2019:
    max_year=rename_19
  else:
    max_year=rename_18
  df_agg.dropna(inplace=True)
  df_agg.sort_values(by=max_year, ascending=True, inplace=True)
  df_agg.tail(15).plot(kind='barh', figsize=(10,10))
  if annotate:
    if df.survey_year.unique().__contains__(2018):
      for index, value in enumerate(df_agg[rename_18].tail(15)): 
        label = format(value, ',')
        plt.annotate(label, xy=(value, index), color='black')
    if df.survey_year.unique().__contains__(2019):
      for index, value in enumerate(df_agg[rename_19].tail(15)): 
        label = format(value, ',')
        plt.annotate(label, xy=(value, index), color='black')
    if df.survey_year.unique().__contains__(2020):
      for index, value in enumerate(df_agg[rename_20].tail(15)): 
        label = format(value, ',')
        plt.annotate(label, xy=(value, index), color='black')
  plt.xlabel(xlabel)
  plt.title(title)
  plt.show()

"""
Used to make comparision of corporate data after agrregating the data over different years.
"""
def type_mix(en, qn, rename, data,den):
  t_mix = cities_data[cities_data['Question Name']==qn]
  df = t_mix[t_mix['Column Name']==en]
  df[['Response Answer']] = df[['Response Answer']].astype('float')
  df_18 = df[df['Year Reported to CDP']==2018]
  df_19 = df[df['Year Reported to CDP']==2019]
  df_20 = df[df['Year Reported to CDP']==2020]
  df_unique = pd.DataFrame([df[df.City==x].values[0] for x in df.City.unique() if len(df[df.City==x].values)>0], 
                            columns=df.columns)[['City', 'Country', 
                                                    'Access', 'Population', 'Population Year', 'Year Reported to CDP', 'City Location']]
  df_18 = df_18[['City', 'Response Answer']].groupby('City').agg('mean')
  df_19 = df_19[['City', 'Response Answer']].groupby('City').agg('mean') 
  df_20 = df_20[['City', 'Response Answer']].groupby('City').agg('mean')
  if rename:
    df_18.rename(columns={'Response Answer': rename+'_18'}, inplace=True)
    df_19.rename(columns={'Response Answer': rename+'_19'}, inplace=True)
    df_20.rename(columns={'Response Answer': rename+'_20'}, inplace=True)

  df = pd.merge(df_18, df_19, on='City', how='left')
  df = pd.merge(df, df_20, on='City', how='left')
  df = pd.merge(df, df_unique, on='City', how='left')
  df.dropna(inplace=True)
  df.index=df.City
  df.drop(columns={'Year Reported to CDP','City'}, inplace=True)
  df = df[df.index.isin(census.city)]
  df = pd.merge(df, census[census.city.isin(df.index)], how='left', left_on='City', right_on='city')
  df.index = df.city
  df.drop(columns={'city'}, inplace=True)
  df['population']=df['population']/1e5
  for col in columns:
    df[col] = df[col]/df['population']
  df[data] = df[data]/den
  df.sort_values(by=data, ascending=True,inplace=True)
  for col in columns:
    df[[data, col]].tail(15).plot(kind='barh', figsize=(10,5))
  plt.xlabel('count')

"""
Used to aggregate categotical data over seperate years.
"""
def type_mix_catg(en,qn,rename,title):
  t_mix = cities_data[cities_data['Question Name']==qn]
  df = t_mix[t_mix['Column Name']==en]
  df.rename(columns={'Response Answer':rename}, inplace=True)
  df_18 = df[df['Year Reported to CDP']==2018]
  df_19 = df[df['Year Reported to CDP']==2019]
  df_20 = df[df['Year Reported to CDP']==2020]
  df_18 = df_18[['City', rename]].groupby(rename).agg('count')
  df_19 = df_19[['City', rename]].groupby(rename).agg('count')
  df_20 = df_20[['City', rename]].groupby(rename).agg('count')
  df_18.rename(columns={'City':'2018'}, inplace=True)
  df_19.rename(columns={'City':'2019'}, inplace=True)
  df_20.rename(columns={'City':'2020'}, inplace=True)
  df = pd.merge(df_18, df_19, on=rename, how='left')
  df = pd.merge(df, df_20, on=rename, how='left')
  df.sort_values(by='2020', ascending=True, inplace=True)
  df.dropna(inplace=True)
  df.tail(15).plot(kind='barh', figsize=(10,7))
  plt.title(title)
  plt.xlabel('count')

"""
Used to aggregate freight data.
"""
def type_mix_freight(en, qn, rename):
  t_mix = cities_data[cities_data['Question Name']==qn]
  df = t_mix[t_mix['Column Name']==en]
  df[['Response Answer']] = df[['Response Answer']].astype('float')
  df_18 = df[df['Year Reported to CDP']==2018]
  df_19 = df[df['Year Reported to CDP']==2019]
  df_20 = df[df['Year Reported to CDP']==2020]
  df_unique = pd.DataFrame([df[df.City==x].values[0] for x in df.City.unique() if len(df[df.City==x].values)>0], 
                            columns=df.columns)[['City', 'Country', 
                                                    'Access', 'Population', 'Population Year', 'Year Reported to CDP', 'City Location']]
  df_18 = df_18[['City', 'Response Answer']].groupby('City').agg('mean')
  df_19 = df_19[['City', 'Response Answer']].groupby('City').agg('mean') 
  df_20 = df_20[['City', 'Response Answer']].groupby('City').agg('mean')
  if rename:
    df_18.rename(columns={'Response Answer': rename+'_18'}, inplace=True)
    df_19.rename(columns={'Response Answer': rename+'_19'}, inplace=True)
    df_20.rename(columns={'Response Answer': rename+'_20'}, inplace=True)

  df = pd.merge(df_18, df_19, on='City', how='left')
  df = pd.merge(df, df_20, on='City', how='left')
  df = pd.merge(df, df_unique, on='City', how='left')
  df.dropna(inplace=True)
  df.index=df.City
  df.drop(columns={'Year Reported to CDP','City'}, inplace=True)
  df = df[df.index.isin(census.city)]
  df = pd.merge(df, census[census.city.isin(df.index)], how='left', left_on='City', right_on='city')
  df.index = df.city
  df.drop(columns={'city'}, inplace=True)
  df['population']=df['population']/1e5
  for col in columns:
    df[col] = df[col]/df['population']
  return df

"""
Used to plot different bar graphs
"""
def corp_wc_type(cn,mn,title,rename):
  df = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name==mn]
  df_sub = df[df.column_name==cn]
  df_sub.rename(columns={'response_value':rename}, inplace=True)
  df_sub.rename(columns={'organization':'2020'}, inplace=True)
  df_sub_agg = df_sub[['2020', rename]].groupby(rename).agg('count')
  df_sub_agg.sort_values(by='2020', ascending=True, inplace=True)
  df_sub_agg.tail(15).plot(kind='barh', figsize=(10,7))
  plt.title(title)
  plt.xlabel('Number of Organization')
  for index, value in enumerate(df_sub_agg['2020'].tail(15)):
    label = format(value, ',')
    plt.annotate(label, xy=(value, index), color='black')
  plt.show()

def corp_wc_risk(cn,mn,title,xlabel,rename):
  df = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name==mn]
  df = df[~df.response_value.isnull()]
  df_sub = df[df.column_name==cn]
  df_sub.rename(columns={'response_value':rename}, inplace=True)
  df_sub_agg = df_sub[df_sub.survey_year==2019][['organization', rename]].groupby(rename).agg('count')
  df_sub_agg.rename(columns={'organization':'2019'}, inplace=True)
  df_sub_agg['2018'] = df_sub[df_sub.survey_year==2018][['organization', rename]].groupby(rename).agg('count')
  df_sub_agg.sort_values(by='2019', ascending=True, inplace=True)
  df_sub_agg.tail(15).plot(kind='barh', figsize=(10,7))
  plt.title(title)
  plt.xlabel(xlabel)
  for index, value in enumerate(df_sub_agg['2019'].tail(15)): 
    label = format(value, ',')
    plt.annotate(label, xy=(value, index), color='black')
  for index, value in enumerate(df_sub_agg['2018'].tail(15)): 
    label = format(value, ',')
    plt.annotate(label, xy=(value, index), color='black')
  plt.show()

def corp_wc_em(cn, mn, title):
  df = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name==mn]
  df_sub = df[df.column_name==cn]
  df_sub['response_value'] = df_sub['response_value'].astype('float')
  df_sub_agg = df_sub[df_sub.survey_year==2018][['organization','response_value']].groupby('organization').agg('mean')
  df_sub_agg.rename(columns={'response_value':'2018'}, inplace=True)
  df_sub_agg['2019'] = df_sub[df_sub.survey_year==2019][['organization','response_value']].groupby('organization').agg('mean')
  df_sub_agg['2020'] = df_sub[df_sub.survey_year==2020][['organization','response_value']].groupby('organization').agg('mean')
  df_sub_agg.dropna(how='any', inplace=True)
  df_sub_agg.sort_values(by='2020', ascending=True, inplace=True)
  df_sub_agg.tail(15).plot(kind='barh', figsize=(10,7))
  plt.title(title)
  plt.xlabel('value')
    
"""
Used to plot map and graphs side by side to compare greenhouse gas emission from 2018 to 2020.
"""  
def greenhouse_em():
  color = ['red', 'green', 'beige', 'orange', 'pink', 'yellow', 'indigo', 'brown', 'gray', 'white']               
  limits = list(greenhouse_location.greenhouse_gas.unique())
  legend_color = {limits[0]:color[0], limits[1]:color[1], limits[2]:color[2], limits[3]:color[3], limits[4]:color[4], limits[5]:color[5], limits[6]:color[6],
                 limits[7]:color[7], limits[8]:color[8], limits[9]:color[9]}
  df_18 = greenhouse_location[greenhouse_location.survey_year==2018][['organization', 'greenhouse_gas']].groupby('greenhouse_gas').agg('count')
  df_18.sort_values(ascending=True, inplace=True, by='organization')
  df_19 = greenhouse_location[greenhouse_location.survey_year==2019][['organization', 'greenhouse_gas']].groupby('greenhouse_gas').agg('count')
  df_19.sort_values(ascending=True, inplace=True, by='organization')
  df_20 = greenhouse_location[greenhouse_location.survey_year==2020][['organization', 'greenhouse_gas']].groupby('greenhouse_gas').agg('count')
  df_20.sort_values(ascending=True, inplace=True, by='organization')

  fig = make_subplots(rows=6, 
                    cols=2, 
                    column_widths=[0.6, 0.4],
                    row_heights=[0.6, 0.1, 0.6, 0.1, 0.6, 0.1],
                    specs=[
                           [{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
                           [            None                    , None],
                           [{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
                           [            None                    , None],
                           [{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
                           [            None                    , None]
                    ],
                    subplot_titles=['2018', '', '2019', '', '2020']
                    )

  for i in range(len(limits)):
    lim = limits[i]
    fig.add_trace(
        go.Scattergeo(
        locationmode = 'ISO-3', 
        lon = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2018').longitude,
        lat = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2018').latitude,
        text = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2018').greenhouse_gas,
        marker = dict(size = 7, color=legend_color[limits[i]]),
        name=lim,
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergeo(
        locationmode = 'ISO-3', 
        lon = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2019').longitude,
        lat = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2019').latitude,
        text = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2019').greenhouse_gas,
        marker = dict(size = 7, color=legend_color[limits[i]]),
        showlegend=False
        ),
        row=3, col=1
    )
    fig.add_trace(
        go.Scattergeo(
        locationmode = 'ISO-3', 
        lon = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2020').longitude,
        lat = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2020').latitude,
        text = greenhouse_location[greenhouse_location.greenhouse_gas==lim].query('survey_year==2020').greenhouse_gas,
        marker = dict(size = 7, color=legend_color[limits[i]]),
        showlegend=False
        ),
        row=5, col=1
    )

    
  fig.add_trace(
    go.Bar(x=df_18.index,y=df_18['organization'], marker=dict(color='crimson'), showlegend=False),
    row=1, col=2
    )
  
  fig.add_trace(
    go.Bar(x=df_19.index,y=df_19['organization'], marker=dict(color='crimson'), showlegend=False),
    row=3, col=2
    )

  fig.add_trace(
    go.Bar(x=df_20.index,y=df_20['organization'], marker=dict(color='crimson'), showlegend=False),
    row=5, col=2
    )

  fig.update_geos(
    resolution=110,
    showocean=True,
    showrivers=True,
    showlakes=True,
    landcolor="white",
    oceancolor="MidnightBlue",
    lakecolor="Blue",
    rivercolor="Blue"
  )

  fig.update_layout(
        template="plotly_dark",
        height=1000,
        width=1000,
        title_text = 'Top 10 common greenhouse gases',
        showlegend = True,
        geo = dict(
            scope = 'world',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

  fig.show()

In [ ]:
emissions_breakdowns = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name=='C7. Emissions breakdowns']
country = emissions_breakdowns[emissions_breakdowns.column_name=='C7.2_C1Country/Region'][['organization', 'response_value']]
country.dropna(inplace=True)
country.rename(columns={'response_value':'country'}, inplace=True)

longitude = pd.read_csv('../input/locations/longitude.csv')
longitude.dropna(inplace=True)
long = []
unique_org_long = pd.DataFrame(longitude.organization.unique(), columns={'organization'})
for i in range(0,len(unique_org_long)):
  long.append(longitude[longitude.organization==unique_org_long.organization[i]]['longitude'].values[0])
unique_org_long['longitude'] = long

latitude = pd.read_csv('../input/locations/latitude.csv')
latitude.dropna(inplace=True)
lat = []
unique_org_lat = pd.DataFrame(latitude.organization.unique(), columns={'organization'})
for i in range(0,len(unique_org_lat)):
  lat.append(latitude[latitude.organization==unique_org_lat.organization[i]]['latitude'].values[0])
unique_org_lat['latitude'] = lat

long_lat = pd.merge(unique_org_long, unique_org_lat, on='organization', how='left')
activity = emissions_breakdowns[emissions_breakdowns.column_name=='C7.3c_C1Activity'][['organization', 'response_value']]
activity.dropna(inplace=True)
activity.rename(columns={'response_value':'activity'}, inplace=True)
greenhouse = emissions_breakdowns[emissions_breakdowns.column_name=='C7.1a_C1Greenhouse gas'][['organization', 'response_value', 'survey_year']]
greenhouse.dropna(inplace=True)
greenhouse.rename(columns={'response_value':'greenhouse_gas'}, inplace=True)
greenhouse['greenhouse_gas'] = [x.split(':')[-1] for x in greenhouse.greenhouse_gas]
city = emissions_breakdowns[emissions_breakdowns.column_name=='C7.3b_C1Facility'][['organization', 'response_value']]
city.dropna(inplace=True)
city.rename(columns={'response_value':'city'}, inplace=True)
business_divission = emissions_breakdowns[emissions_breakdowns.column_name=='C7.6a_C1Business division'][['organization', 'response_value']]
business_divission.dropna(inplace=True)
business_divission.rename(columns={'response_value':'business_divission'}, inplace=True)

top_activities = activity.activity.value_counts()[0:10]
top_greenhouse_gases = greenhouse.greenhouse_gas.value_counts()[0:10]
top_business_divisions = business_divission.business_divission.value_counts()[0:10]

activity_df = activity[activity.activity.isin(top_activities.keys())]
greenhouse_df = greenhouse[greenhouse.greenhouse_gas.isin(top_greenhouse_gases.keys())]

greenhouse_location = pd.merge(greenhouse_df, long_lat, on='organization', how='left')
greenhouse_location.dropna(inplace=True)
activity_location = pd.merge(activity_df, long_lat, on='organization', how='left')
activity_location.dropna(inplace=True)
gas_count = pd.DataFrame(top_greenhouse_gases)
gas_count.rename(columns={'greenhouse_gas':'count'}, inplace=True)
gas_count['greenhouse_gas'] = gas_count.index
greenhouse_location = pd.merge(greenhouse_location, gas_count, on='greenhouse_gas',how='left')

# Part 1: Key performance indicators (KPI) that relate to the environmental and social issues

#### How were these KPI calculated:
The methodology has been derived from an analysis of the impact that the       organization's business have on the environment. The analysis takes into account the response values and remarks given in the data files. These include data like types of activities, emmision figures, social and environmental factors. Based on the questions answered, following KPI have been included:

## 1.1. Emissions to air

### a) Co2 Emission Factor

In [ ]:
corp_wc_yr('C8.2d_C1Emission factor','C8. Energy','Emission factor','Highest 15 CO2 emmission factors (from data with both values)','2018','2019','2020',annotate=False)

It seems, among the companies with highest factor value, there was a decrease from 2018 to 2019.

### b) Change in supplier-related, customer-related and overall scope 3 emissions 
 

### With % decrease in number of suppliers

In [ ]:
def corp_wc_df(cn, mn, rename_sub):
  df = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name==mn]
  df_sub = df[df.column_name==cn][['organization','response_value']]
  df_sub['response_value'] = df_sub['response_value'].astype('float')
  df_sub = df_sub.groupby('organization').agg('mean')
  df_sub.rename(columns={'response_value':rename_sub}, inplace=True)
  return df_sub

supplier_emmission = corp_wc_df('C12.1a_C5% of supplier-related Scope 3 emissions as reported in C6.5', 'C12. Engagement',
                                '% of supplier-related Scope 3 emissions')
customer_emmission = corp_wc_df('C12.1b_C4% of customer - related Scope 3 emissions as reported in C6.5', 'C12. Engagement',
                                '% of customer-related Scope 3 emissions')
total_emmission = corp_wc_df('C12.1a_C5% Scope 3 emissions as reported in C6.5', 'C12. Engagement','% Scope 3 emissions')

supplier = corp_wc_df('C12.1a_C3% of suppliers by number', 'C12. Engagement','% of suppliers by number')
customer = corp_wc_df('C12.1b_C3% of customers by number', 'C12. Engagement','% of customers by number')
procurement = corp_wc_df('C12.1a_C4% total procurement spend (direct and indirect)', 'C12. Engagement',
                         '% total procurement spend (direct and indirect)')

emmissions = pd.merge(customer_emmission, supplier_emmission, on='organization', how='left')
emmissions = pd.merge(emmissions, total_emmission, on='organization', how='left')
emmissions.dropna(how='any', inplace=True)

supplier_effect = pd.merge(supplier, emmissions, on='organization', how='right')
customer_effect = pd.merge(customer, emmissions, on='organization', how='right')
procurement_effect = pd.merge(procurement, emmissions, on='organization', how='right')

supplier_effect.sort_values(by='% of suppliers by number', ascending=True, inplace=True)
supplier_top = supplier_effect[['% of suppliers by number',
                                '% of supplier-related Scope 3 emissions', 
                                '% of customer-related Scope 3 emissions',
                                '% Scope 3 emissions'
                                ]].iloc[::10,:].head(15)
supplier_top.plot(kind='barh', figsize=(15,10))
plt.title('Change in Emmissions with change in % of suppliers by number')
plt.xlabel('value')
plt.show()


As the % of suppliers by number decrease, there seems to be a decrease in % of customer related scope 3 emissions. Though % of supplier related and overall scope 3 emissions become more extreme.

### With  % decrease in number of customers

In [ ]:
customer_effect.sort_values(by='% of customers by number', ascending=True, inplace=True)
customer_top = customer_effect[['% of customers by number',
                                '% of supplier-related Scope 3 emissions', 
                                '% of customer-related Scope 3 emissions',
                                '% Scope 3 emissions'
                                ]].iloc[::10,:].head(15)
customer_top.plot(kind='barh', figsize=(15,10))
plt.title('Change in Emmissions with change in % of customers by number')
plt.xlabel('value')
plt.show()

Again, as the % of customers by number decrease, there seems to be a decrease in % of customer related scope 3 emissions. Though % of supplier related and overall scope 3 emissions become more extreme.

### With % increase in total procurement spend (direct and indirect)

In [ ]:
procurement_effect.sort_values(by='% total procurement spend (direct and indirect)', ascending=True, inplace=True)
supplier_top = procurement_effect[['% total procurement spend (direct and indirect)',
                                '% of supplier-related Scope 3 emissions', 
                                '% of customer-related Scope 3 emissions',
                                '% Scope 3 emissions'
                                ]].iloc[::10,:].head(15)
supplier_top.plot(kind='barh', figsize=(15,10))
plt.title('Change in Emmissions with change in % total procurement spend (direct and indirect)')
plt.xlabel('value')
plt.show()

As the % total procurement spend (direct and indirect) decrease, the % of customer related scope 3 emissions increase, whereas the overall and suppliers related % scope 3 emissions seem to have become somewhat less extreme.

### c) Gross global Scope 1 emissions (metric tons CO2e)

In [ ]:
corp_wc_em('C6.1_C1Gross global Scope 1 emissions (metric tons CO2e)','C6. Emissions data',
           'Top 15 organizations w.r.t Gross global Scope 1 emissions (metric tons CO2e)')

Overall, there seems to be a decrease in the gross scope 1 emissions of metric tons Co2 in year 2020 compared to 2018 and 2019.

### d) Gross global Scope 2 market-based emissions

In [ ]:
corp_wc_em('C6.3_C2Scope 2, market-based (if applicable)','C6. Emissions data',
           'Top 15 organizations w.r.t Gross global Scope 2 market-based emissions')

Again, there seems to be a decrease in the gross scope 2 market based emissions in year 2020 compared to 2018 and 2019.

### e) Gross global Scope 2 location-based emissions

In [ ]:
corp_wc_em('C7.5_C2Scope 2, location-based (metric tons CO2e)','C7. Emissions breakdowns',
           'Top 15 organizations w.r.t Gross global Scope 2 location-based emissions')

Baring a few extreme values, the trend seems to be the same, gross global scope 2 location based emissions have also decreased in 2020.

## 1.2. Resources consumed

### a) Change In Overall MWh Power

In [ ]:
def corp_wc_power(cn, mn, rename_sub):
  df = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name==mn]
  df_sub = df[df.column_name==cn][['organization', 'response_value', 'survey_year']]
  df_sub['response_value'] = df_sub['response_value'].astype('float')
  df_sub_agg = pd.DataFrame(columns={'2018','2019'})
  df_sub_agg['2018'] = df_sub[df_sub.survey_year==2018][['organization','response_value']].groupby(['organization']).agg('mean')['response_value']
  df_sub_agg['2019'] = df_sub[df_sub.survey_year==2019][['organization','response_value']].groupby(['organization']).agg('mean')['response_value']
  df_sub_agg[rename_sub] = df_sub_agg['2019'] - df_sub_agg['2018']
  return df_sub_agg
  
mwh_gross = corp_wc_power('C8.2a_C4Total MWh','C8. Energy','gross_change')
mwh_renewable = corp_wc_power('C8.2a_C2MWh from renewable sources','C8. Energy','renewable_change')
mwh_non_renewable = corp_wc_power('C8.2a_C3MWh from non-renewable sources','C8. Energy','non_renewable_change')
mwh = pd.merge(mwh_gross, mwh_renewable, on='organization', how='right')
mwh = pd.merge(mwh, mwh_non_renewable, on='organization', how='left')
mwh.dropna(how='any', inplace=True, axis=0)

mwh.sort_values(by='gross_change', ascending=True, inplace=True)
mwh[['gross_change', 'renewable_change', 'non_renewable_change']].tail(15).plot(kind='barh', figsize=(15,7))
plt.xlabel('Response Value')
plt.title('Top 15 Organizations Based On Increase In Overall MWh Power')
plt.show()

Organizations, where overall MWh power use has increased the most from 2018 to 2019, have depended more on non-renewable source of energy to fulfil there demands.

### b) Change in Renewable MWh Power

In [ ]:
mwh.sort_values(by='renewable_change', ascending=True, inplace=True)
mwh[['gross_change', 'renewable_change', 'non_renewable_change']].tail(15).plot(kind='barh', figsize=(15,7))
plt.xlabel('Response Value')
plt.title('Top 15 Organizations Based On Increase In Renewable MWh Power')
plt.show()

In organizations, where maximum MWh power was obtained from renewable source of energy, the overall power consumption has either decreased from 2018 to 2019, or was very less.

### c) Change in Non-Renewable MWh Power

In [ ]:
mwh.sort_values(by='non_renewable_change', ascending=True, inplace=True)
mwh[['gross_change', 'renewable_change', 'non_renewable_change']].tail(15).plot(kind='barh', figsize=(15,7))
plt.xlabel('Response Value')
plt.title('Top 15 Organizations Based On Increase In Non-Renewable MWh Power')
plt.show()

Again, in organizations where the use of non-renewable source of energy was highest, were consuming maximum MWh overall power.

## 1.3. Risks and Opportunities

### a) Increase in Risk Type

In [ ]:
corp_wc_risk('C2.3a_C3Risk type','C2. Risks and opportunities','Increase in Risk Type','count','Risk Types')

Both transitional and physical risk have increased for organizations from 2018 to 2019.

### b) Potential Financial Impacts in 2019

In [ ]:
risk_op = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name=='C2. Risks and opportunities'][['organization', 'column_name', 'survey_year', 'response_value', 'comments']]
risk_op = risk_op[~risk_op.response_value.isnull()]

financial_impact_min = risk_op[risk_op.column_name=='C2.3a_C12Potential financial impact figure â€“ minimum (currency)']
financial_impact_min.rename(columns={'response_value':'Potential minimum financial impact'}, inplace=True)
financial_impact_min['Potential minimum financial impact'] = financial_impact_min['Potential minimum financial impact'].astype('float')
financial_impact_min_df = financial_impact_min[['organization', 'Potential minimum financial impact']].groupby('organization').agg('mean')

financial_impact_max = risk_op[risk_op.column_name=='C2.3a_C12Potential financial impact figure â€“ minimum (currency)']
financial_impact_max.rename(columns={'response_value':'Potential maximum financial impact'}, inplace=True)
financial_impact_max['Potential maximum financial impact'] = financial_impact_max['Potential maximum financial impact'].astype('float')
financial_impact_max_df = financial_impact_max[['organization', 'Potential maximum financial impact']].groupby('organization').agg('mean')

financial_impact_all = pd.merge(financial_impact_min_df, financial_impact_max_df, on='organization', how='left')
financial_impact_all.dropna(how='any', inplace=True)
financial_impact_all.sort_values(by='Potential maximum financial impact', ascending=True, inplace=True)

financial_impact_all.tail(15).plot(kind='barh', figsize=(15,10))
plt.xlabel('Financial Impact')
plt.title('Some Potential Financial Impacts in 2019')
for index, value in enumerate(financial_impact_all['Potential minimum financial impact'].tail(15)): 
    label = format(value, ',')
    plt.annotate(label, xy=(value, index), color='black')
for index, value in enumerate(financial_impact_all['Potential maximum financial impact'].tail(15)): 
    label = format(value, ',')
    plt.annotate(label, xy=(value, index), color='black')
plt.show()

Potential financial impact seems to be highest in banking and energy related sectors.

### c) Counts of various impact levels

In [ ]:
corp_wc_risk('C2.3a_C9Magnitude of impact','C2. Risks and opportunities','Counts of various impact levels','count','Impact Levels')

In almost all categories, magnitude of impact has increased from 2018 to 2019. Medium level impacts have increased the most.

### d) Cases of various types of Opportunity

In [ ]:
corp_wc_risk('C2.4a_C3Opportunity type','C2. Risks and opportunities','Cases of various types of Opportunity','count','Opportunity Types')

Opportunities have also increased from 2018 to 2019. Interestingly, resource efficiency opportunity has the highest count.

### e) Time horizon level counts

In [ ]:
corp_wc_risk('C2.3a_C7Time horizon','C2. Risks and opportunities','Time horizon level counts','count','Horizen Levels')

Again the count has increase in 2019 from 2018. Medium-term time horizen is the most common.

## 1.4. Targets and performance

### a) KPI measure at base and target level

In [ ]:
tg_pr = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name=='C4. Targets and performance'][['organization', 'column_name', 'survey_year', 'response_value', 'comments']]

kpi_base = tg_pr[tg_pr.column_name=='C4.2_C7KPI in baseline year']
kpi_target = tg_pr[tg_pr.column_name=='C4.2_C8KPI in target year']
kpi_base['response_value'] = kpi_base['response_value'].astype('float')
kpi_target['response_value'] = kpi_target['response_value'].astype('float')
kpi_base_agg = kpi_base[['organization', 'response_value']].groupby('organization').agg('mean')
kpi_target_agg = kpi_target[['organization', 'response_value']].groupby('organization').agg('mean')
kpi_target_agg.dropna(how='any', inplace=True)
kpi_base_agg.dropna(how='any', inplace=True)
kpi_base_agg.rename(columns={'response_value':'KPI Base'}, inplace=True)
kpi_target_agg.rename(columns={'response_value':'KPI Target'}, inplace=True)
kpi = pd.merge(kpi_base_agg, kpi_target_agg, on='organization', how='left')
kpi.dropna(how='any', inplace=True)
kpi.sort_values(by='KPI Target', ascending=True, inplace=True)
kpi.tail(10).plot(kind='barh', figsize=(10,5))
plt.title('KPI measure at base and target level')
plt.xlabel('value')

KPI in target year seems to have increased in most cases.

### b) Revenue from low carbon product(s) in the reporting year

In [ ]:
revenue_low_carbon = tg_pr[tg_pr.column_name=='C4.5a_C5% revenue from low carbon product(s) in the reporting year']
revenue_low_carbon['response_value'] = revenue_low_carbon['response_value'].astype('float')
revenue_low_carbon_agg = revenue_low_carbon[revenue_low_carbon.survey_year==2018][['organization','response_value']].groupby('organization').agg('mean')
revenue_low_carbon_agg.rename(columns={'response_value':'% revenue in 2018'}, inplace=True)
revenue_low_carbon_agg['% revenue in 2019'] = revenue_low_carbon[revenue_low_carbon.survey_year==2019][['organization','response_value']].groupby('organization').agg('mean')
revenue_low_carbon_agg['% revenue in 2020'] = revenue_low_carbon[revenue_low_carbon.survey_year==2020][['organization','response_value']].groupby('organization').agg('mean')
revenue_low_carbon_agg.dropna(how='any', inplace=True)
revenue_low_carbon_agg.sort_values(by='% revenue in 2020', ascending=True, inplace=True)
revenue_low_carbon_agg.iloc[::10,:].tail(10).plot(kind='barh', figsize=(10,5))
plt.title('% Revenue from low carbon product(s) in the reporting year')
plt.xlabel('value')

Numbers seems to be confusing here, it might become more clear from long term data.

### c) Estimated annual CO2e savings (metric tonnes CO2e) in 2020

In [ ]:
co2_saving = tg_pr[tg_pr.column_name=='C4.3b_C2Estimated annual CO2e savings (metric tonnes CO2e)']
co2_saving['response_value'] = co2_saving['response_value'].astype('float')
co2_saving.rename(columns={'response_value':'2020'}, inplace=True)
co2_saving_agg = co2_saving[['organization', '2020']].groupby('organization').agg('mean')
co2_saving_agg.sort_values(by='2020', ascending=True, inplace=True)
co2_saving_agg.dropna(how='any', inplace=True)
co2_saving_agg.tail(10).plot(kind='barh', figsize=(10,5))
plt.title('Top 10 Estimated annual CO2e savings (metric tonnes CO2e) in 2020')
plt.xlabel('Savings')
for index, value in enumerate(co2_saving_agg['2020'].tail(10)):
  label = format(value, ',')
  plt.annotate(label, xy=(value, index), color='black')
plt.show()

As the data only consisted values from 2020, its hard to make a conclusion in terms of improvement. But overall, there seems to be a considerable amount savings.

# Part 2: Intersection between environmental issues and social issues

### 2.1. Increase in the emission of greenhouse gases

In [ ]:
greenhouse_em()

It can be seen that the volume of most common greenhouse gases have increase from 2018 to 2019 and from 2019 to 2020.

### 2.2. Emissions reduction (metric tonnes CO2e)

In [ ]:
type_mix('Estimated emissions reduction (metric tonnes CO2e)', 'What actions is your city taking to reduce emissions? \
Please also indicate estimated emissions reduction potential and status of the emissions reduction actions your city has planned.', 
'Co2 reduction', 'Co2 reduction_20',5e5)

In every single comparision, cases of illness per unit population have increased as Co2e emission reduction reduces.

### 2.3. Annual distance travelled (kms)

In [ ]:
type_mix('Average annual distance travelled (kms)', 'What is the average \
annual distance travelled by each transport mode in your city (kms)?', 'distance','distance_20',1e9)

Again, as more and more distance is travelled, cases of illness per unit population increase.

### 2.4. Percentage of Wastewater collected

In [ ]:
type_mix('Percentage of wastewater collected', 'Please provide \
the percentage breakdown of the wastewater your city collects and the percentage breakdown of the treatment type for each wastewater type:', 
'wastewater','wastewater_20',1e2)

Though its a mixed result, but in some cases increase in wastewater collected has led to more cases of illness. It could be due to more power consumption for such a process.

### 2.5. Difference in amount of electricity consumed from polluting and non polluting sources

In [ ]:
def type_mix_energy(en, qn, rename):
  t_mix = cities_data[cities_data['Question Name']==qn]
  df = t_mix[t_mix['Column Name']==en]
  df[['Response Answer']] = df[['Response Answer']].astype('float')
  df_18 = df[df['Year Reported to CDP']==2018]
  df_19 = df[df['Year Reported to CDP']==2019]
  df_20 = df[df['Year Reported to CDP']==2020]
  df_unique = pd.DataFrame([df[df.City==x].values[0] for x in df.City.unique() if len(df[df.City==x].values)>0], 
                            columns=df.columns)[['City', 'Country', 
                                                    'Access', 'Population', 'Population Year', 'Year Reported to CDP', 'City Location']]
  df_18 = df_18[['City', 'Response Answer']].groupby('City').agg('mean')
  df_19 = df_19[['City', 'Response Answer']].groupby('City').agg('mean') 
  df_20 = df_20[['City', 'Response Answer']].groupby('City').agg('mean')
  if rename:
    df_18.rename(columns={'Response Answer': rename+'_18'}, inplace=True)
    df_19.rename(columns={'Response Answer': rename+'_19'}, inplace=True)
    df_20.rename(columns={'Response Answer': rename+'_20'}, inplace=True)

  df = pd.merge(df_18, df_19, on='City', how='left')
  df = pd.merge(df, df_20, on='City', how='left')
  df = pd.merge(df, df_unique, on='City', how='left')
  df.sort_values(by=rename+'_20', ascending=True, inplace=True)
  df.dropna(inplace=True)
  df.index=df.City
  df.drop(columns={'Year Reported to CDP','City'}, inplace=True)
  return df
solar = type_mix_energy('Solar','Please indicate the energy mix of electricity consumed in your city.','solar')
solar.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
coal = type_mix_energy('Coal','Please indicate the energy mix of electricity consumed in your city.','coal')
coal.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
oil = type_mix_energy('Oil','Please indicate the energy mix of electricity consumed in your city.','oil')
oil.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
biomass = type_mix_energy('Biomass','Please indicate the energy mix of electricity consumed in your city.','biomass')
biomass.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
geothermal = type_mix_energy('Geothermal','Please indicate the energy mix of electricity consumed in your city.','geothermal')
geothermal.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
nuclear = type_mix_energy('Nuclear','Please indicate the energy mix of electricity consumed in your city.','nuclear')
nuclear.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
hydro = type_mix_energy('Hydro','Please indicate the energy mix of electricity consumed in your city.','hydro')
wind = type_mix_energy('Wind','Please indicate the energy mix of electricity consumed in your city.','wind')
wind.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
gas = type_mix_energy('Gas','Please indicate the energy mix of electricity consumed in your city.','gas')
gas.drop(columns={'Country', 'Access', 'Population', 'Population Year', 'City Location'}, inplace=True)
energy = pd.merge(hydro,solar,on='City',how='left')
energy = pd.merge(energy,coal,on='City',how='left')
energy = pd.merge(energy,oil,on='City',how='left')
energy = pd.merge(energy,biomass,on='City',how='left')
energy = pd.merge(energy,geothermal,on='City',how='left')
energy = pd.merge(energy,nuclear,on='City',how='left')
energy = pd.merge(energy,wind,on='City',how='left')
energy = pd.merge(energy,gas,on='City',how='left')
energy['safe_energy_18'] = energy['solar_18']+energy['hydro_18']+energy['geothermal_18']+energy['wind_18']+energy['nuclear_18']
energy['safe_energy_19'] = energy['solar_19']+energy['hydro_19']+energy['geothermal_19']+energy['wind_19']+energy['nuclear_19']
energy['safe_energy_20'] = energy['solar_20']+energy['hydro_20']+energy['geothermal_20']+energy['wind_20']+energy['nuclear_20']                                                                                                                                                                                              
energy['unsafe_energy_18'] = energy['coal_18']+energy['biomass_18']+energy['gas_18']+energy['oil_18']
energy['unsafe_energy_19'] = energy['coal_19']+energy['biomass_19']+energy['gas_19']+energy['oil_19']
energy['unsafe_energy_20'] = energy['coal_20']+energy['biomass_20']+energy['gas_20']+energy['oil_20']
energy = energy[['safe_energy_18', 'safe_energy_19', 'safe_energy_20', 'unsafe_energy_18', 'unsafe_energy_19', 'unsafe_energy_20']]
energy.dropna(inplace=True)
energy = energy[energy.index.isin(census.city)]
energy = pd.merge(energy, census[census.city.isin(energy.index)], how='left', left_on='City', right_on='city')
energy['population'] = energy['population']/1e5
energy['safe_unsafe_diff_per_head'] = (energy['safe_energy_20'] - energy['unsafe_energy_20'] + 100)/energy['population']
energy.index = energy.city
energy.drop(columns={'city'}, inplace=True)
energy.sort_values(by='safe_unsafe_diff_per_head', ascending=True,inplace=True)
for col in columns:
  energy[['safe_unsafe_diff_per_head', col]].tail(15).plot(kind='barh',figsize=(10,5))

It can be seen that some illness like cancer, diabetes, kidney related, mental health relates and sleep disorder have increased when more unsafe source of energy (from fossil fuel) are used.

### 2.6. Bus fleet size

In [ ]:
buses = type_mix('Number of buses', 'Please provide the total fleet size and number of \
vehicle types for the following modes of transport:', 'buses', 'buses_20',1e2)

Overall as the fleet size of public transport has reduced, cases of illness per unit population have increased

### 2.7. Number of freight vehicles

In [ ]:
freight = type_mix_freight('Number of freight vehicles', 'Please provide the total fleet size and number of \
vehicle types for the following modes of transport:', 'freight')
freight['freight_20'] = freight['freight_20']/1e2
freight = freight[freight.index!='Athens']
freight.sort_values(by='freight_20', ascending=True, inplace=True)
for col in columns:
  freight[['freight_20', col]].tail(20).plot(kind='barh', figsize=(10,5))
  plt.xlabel('count')

As the volume of freight vehicles have increased, so have cases of illness.

### 2.8 Cases of various types of risks

In [ ]:
corp_wc_risk('C2.3a_C4Primary climate-related risk driver','C2. Risks and opportunities','Cases of various types of risks','count','Risk Types')

Weather related acute and chronic risks are the most common types.

# Part 3: Do city and corporate ambitions take social and environmental factors into account ?

### 3.1. Climate Hazards

In [ ]:
type_mix_catg('Climate Hazards',
              'Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity.  Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.',
              'hazard','Reduction in city count for various hazards')
type_mix_catg('Probability of hazard', 'Please list the most significant climate hazards faced by your city and indicate the probability and consequence of these hazards, as well as the expected future change in frequency and intensity.  Please also select the top 3 assets or services that are affected by the climate hazard, and provide a description of the impact.', 
              'hazard_probability', 'Reduction in city count for various levels hazards')

There seems to be a lot of good work done at city level, when cases of various hazards and levels of hazards are seen. Both have reduced from 2018 to 2020.

### 3.2. Presence of organizations

In [ ]:
org_count = org[['organization', 'address_city']].groupby('address_city').agg('count')
p = pd.DataFrame([org[org.address_city==x].values[0] for x in org.address_city.unique()], columns=org.columns)[['address_city', 'population']]
org_count = pd.merge(org_count, p, how='left', on='address_city')
org_count.index = org_count.address_city
org_count.drop(columns={'address_city'}, inplace=True)
org_count.sort_values(by='organization', ascending=True, inplace=True)
org_count['population']=org_count['population']/1e4
org_count.iloc[::2,:].plot(kind='barh', figsize=(10,10))
plt.title('Decrease in number of organization per unit population')
plt.xlabel('count')

At the organizational level, a lot of works needs to be done in terms of organizational presence as the organizations seem be be present only at limited locations. This might be the cause for increase in unemployment rate.

### 3.3. Growth In Primary Industry Categories

In [ ]:
df = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name=='C8. Energy'][['organization', 'question_unique_reference', 'module_name', 'comments', 'survey_year', 'column_name', 'response_value']]

primary_industry_18 = Corporates_Disclosing_to_CDP[Corporates_Disclosing_to_CDP.survey_year==2018][['organization', 'primary_industry']].groupby(['primary_industry']).agg(['count'])
primary_industry_18.columns = ['count in 2018']

primary_industry_19 = Corporates_Disclosing_to_CDP[Corporates_Disclosing_to_CDP.survey_year==2019][['organization', 'primary_industry']].groupby(['primary_industry']).agg(['count'])
primary_industry_19.columns = ['count in 2019']

primary_industry_20 = Corporates_Disclosing_to_CDP[Corporates_Disclosing_to_CDP.survey_year==2020][['organization', 'primary_industry']].groupby(['primary_industry']).agg(['count'])
primary_industry_20.columns = ['count in 2020']

primary_industry = pd.merge(primary_industry_18, primary_industry_19, on='primary_industry', how='left')
primary_industry = pd.merge(primary_industry, primary_industry_20, on='primary_industry', how='left')

primary_industry.transpose().plot(kind='line', figsize=(15, 7))
plt.title('Growth In Primary Industry Categories')
plt.ylabel('count')

Apparel and manufacturing have expanded the most. While industries like Biotech, health and pharma seem to be controlled by a small group as there seem to be very less expansion. This might explain the sky rising price of healthcare service and medicine.

### 3.4. Corporates Disclosing to CDP

In [ ]:
year_org_count = Corporates_Disclosing_to_CDP[['organization', 'survey_year']].groupby(['survey_year']).agg(['count'])
year_org_count.columns = ['organizations']
year_org_count.plot(kind='barh', figsize=(10, 4))
plt.title('Corporates Disclosing to CDP')
plt.xlabel('count')

Overall, there seems to be a positive attitude among corporations, as participation has increased from 2018 to 2020.

### 3.5. Efforts Towards Reducing Emmissions

In [ ]:
comments = df[df.question_unique_reference=='State how much fuel in MWh your organization has consumed (excluding feedstocks) by fuel type.'][['organization', 'comments', 'survey_year']]
comments.dropna(inplace=True, how='any')
energy_source = []
non_ren = re.compile('|'.join(['diesel', 'Diesel', 'Kerosene', 'kerosene', 'Gasolene', 'gasolene', 'Gasoline/petrol', 'Coal', 'coal', 'oil', 'Oil']))
for i in range(0,len(comments)):
  if non_ren.search(comments.comments.iloc[i]):
    energy_source.append('No effort made')
  else:
    energy_source.append('Some effort made')

comments['Level of Effort'] = energy_source
response_index = [i for i in comments[comments['Level of Effort']=='Some effort made']['comments'].index]

no_response = re.compile('|'.join(['gas', 'Gas', 'LPG']))
for i in response_index:
  if no_response.search(comments[comments.index==i].comments.values[0]):
    comments['Level of Effort'][i] = 'Some Effort'
  else:
    comments['Level of Effort'][i] = 'No Response'

effort = comments[['organization', 'Level of Effort']].groupby(['Level of Effort']).agg('count')
effort.rename(columns={'organization':'2018'}, inplace=True)
effort.plot(kind='barh', figsize = (10,5))
valid = comments.comments.nunique()
plt.title('Efforts Towards Reducing Emmissions (from {} valid comments)'.format(valid))
plt.xlabel('count')  
for index, value in enumerate(effort['2018']): 
    label = format(value, ',')
    plt.annotate(label, xy=(value, index), color='black')
plt.show()

Some effort at organizational level has been done by avoiding fuel based energy source.

### 3.6. Organization undertaking energy-related activity

In [ ]:
initiative = df[df.column_name=='C8.2_C1Indicate whether your organization undertakes this energy-related activity'][['organization', 'response_value', 'survey_year']]
initiative['response_value'] = initiative['response_value'].astype('str')
initiative['2018'] = initiative[initiative.survey_year==2018]['survey_year']
initiative['2019'] = initiative[initiative.survey_year==2019]['survey_year']
in_df = initiative[['2018', '2019', 'response_value']].groupby(['response_value']).agg('count')
in_df['Percentage Increment '] = (in_df['2019'] - in_df['2018'])/100
print('\nIndicate whether your organization undertakes this energy-related activity')
in_df

Looking at the percentage increase in count of organizations who have participated, the count has increased more for those who have not.

### 3.7. Number of initiatives

In [ ]:
tg_pr = Corporates_Climate_Water_Dataset[Corporates_Climate_Water_Dataset.module_name=='C4. Targets and performance'][['organization', 'column_name', 'survey_year', 'response_value', 'comments']]

initiative = tg_pr[tg_pr.column_name=='C4.3a_C1Number of initiatives']
initiative = initiative[~initiative.response_value.isnull()]
initiative = initiative[initiative.response_value!='Yes']
initiative = initiative[initiative.response_value!='No']
initiative.response_value = initiative.response_value.astype('float')
initiative_agg = initiative[initiative.survey_year==2019][['organization', 'response_value']].groupby('organization').agg('mean')
initiative_agg.rename(columns={'response_value':'2019'}, inplace=True)
initiative_agg['2019'] = initiative_agg['2019'].astype('int')
initiative_agg['2020'] = initiative[initiative.survey_year==2020][['organization', 'response_value']].groupby('organization').agg('mean')
initiative_agg = initiative_agg[~initiative_agg['2020'].isnull()]
initiative_agg['2020'] = initiative_agg['2020'].astype('int')
initiative_agg.sort_values(by='2020', ascending=True, inplace=True)
initiative_agg.tail(10).plot(kind='barh', figsize=(15,5))
plt.title('Number of initiatives')
plt.xlabel('count')

Number of initiatives from organizations have increase from 2019 to 2020.

### 3.8. Activity Types

In [ ]:
corp_wc_type('C4.3b_C1Activity type','C4. Targets and performance','Top 15 Corporate Activity Types in year 2020','Activity Type')

Energy efficiency type activities semm to be the most common in 2020.

### 3.9. Initiative Types

In [ ]:
corp_wc_type('C4.3b_C1Initiative category & Initiative type','C4. Targets and performance','Top 15 Corporate Initiative Types in year 2020',
             'Initiative Type')

Power related activity types like lightning, heating, ventilation, air conditioning and process optimization are the most common corporate activity type in 2020.

### 3.10. Target Type

In [ ]:
corp_wc_type('C4.2b_C5Target type: category & Metric (target numerator if reporting an intensity target)_G','C4. Targets and performance',
             'Top 15 Corporate Target Types in year 2020','Target Type')

Again energy consuption or efficiency and waste management are the most common corporate activity type in 2020.